## Task A: Left Ventricle Segmentation

In [ ]:
import numpy as np
import Segmentation as Seg


for patientNumber in range(1,11):
    print(patientNumber)
    for title in ['2CH', '4CH']:
        output = []
        image, aspect, spacing = Seg.read_patient_mhd_file(patientNumber, title + "_sequence", test=True)
        for i in range(len(image)):
            b, region = Seg.segmentImage(image[i], aspect, title, False, False)
            region[region>0] = 1
            output.append(region)

        Seg.write_mhd_file(np.array(output), patientNumber, title, spacing)

## Task B: 3D Visualization

In [ ]:
import numpy as np
import pyvista as pv
import Reconstruction as Rec
import Segmentation as Seg


for n in range(1,4): # first 3 cases for videos
    TwoSequence, FourSequence, aspectRatio = Seg.getSequenceSegmentations(n, True)

    newTwo = Rec.resampleAllContours(TwoSequence)
    newFour = Rec.resampleAllContours(FourSequence)

    interpTwo = Rec.interpolateSequence(newTwo, 60)
    interpFour = Rec.interpolateSequence(newFour, 60)

    frames = []
    for i in range(len(interpTwo)):
        frames.append(Rec.generatePointCloud(interpTwo[i], interpFour[i], 3, aspectRatio, i == 0))

    plotter = pv.Plotter(shape=(1, 2))
    plotter.open_movie(f"staticMovie{n}.mp4")

    reps = 10

    for i in range(reps):
        for frame in frames:
            plotter.clear()
            
            p = pv.wrap(np.array(frame))
            surf = p.reconstruct_surface()

            plotter.subplot(0, 0)
            plotter.add_mesh(p, style='points', point_size=3.0)
            plotter.add_title('Point Cloud')
            plotter.subplot(0, 1)
            plotter.add_mesh(surf, opacity = 0.4, show_edges=True)
            plotter.add_title('Surface')
            
            plotter.write_frame()  # Write this frame
            
    plotter.close()

## Task C: Stroke Volume and Ejection Fraction Calculation

In [3]:
import Reconstruction as Rec
import Segmentation as Seg
import pyvista as pv
import numpy as np
import re


opt1 = [2.15060547e-09, 1.48584167e+00, 8.28364558e+01]
opt2 = [3.76830004e-05, 8.95933170e-01, 3.36601220e+01]

def regression(x, a, b, c):
    return a * np.exp(b * x) + c

def getVolumesForTest(patient):
    types = ['2CH_sequence', '4CH_sequence']
    contours = []
    aspectRatio = 0
    for t in types: 
        image, aspect, spacing = Seg.read_patient_mhd_file(patient, t, test=True)
        
        b, region = Seg.segmentImage(image[0], aspect, t, False, False)
        contours.append(region.astype(np.uint8))
        
        b, region = Seg.segmentImage(image[-1], aspect, t, False, False)
        contours.append(region.astype(np.uint8))
        if aspect != aspectRatio and aspectRatio > 0:
            raise Exception('Different Aspects')
        aspectRatio = aspect
        
        
    resampled = Rec.resampleAllContours(contours)

    ED = Rec.generatePointCloud(resampled[0], resampled[2], 4, aspectRatio, False)
    ES = Rec.generatePointCloud(resampled[1], resampled[3], 4, aspectRatio, False)
    
    EDsurface = pv.wrap(np.array(ED)).reconstruct_surface().volume
    ESsurface = pv.wrap(np.array(ES)).reconstruct_surface().volume

    EDreal = regression(np.log(EDsurface), *opt1)
    ESreal = regression(np.log(ESsurface), *opt2)
   
    return EDreal, ESreal, (EDreal - ESreal) / EDreal
    

for i in range(1, 11):
    EDreal, ESreal, EFrac = getVolumesForTest(i)
    strokeVol = EDreal - ESreal
    with open(Seg.RESULT_PATH + f"Test{i}/taskC.txt", "w") as f:
        f.write(str(strokeVol) + '\n' + str(EFrac))

/Users/pgonzales/miniforge3/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/pgonzales/miniforge3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
